In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

# TODO: Fill in the Google Drive path where you uploaded the assignment
# Example: If you create a 2020FA folder and put all the files under A1 folder, then '2020FA/A1'
# GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = '2020FA/A1'
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = '/content/drive/MyDrive/Colab Notebooks/24年暑假科研/Skin_Conditions'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

['.DS_Store', 'Eczema', 'Carcinoma', 'Acne', 'Milia', 'Keratosis', 'Rosacea']


In [4]:
import numpy as np
import pandas as pd

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

dataset_dir = 'Skin_Conditions'
dataset = image_dataset_from_directory(
    GOOGLE_DRIVE_PATH,
    labels='inferred',
    label_mode='int',
    image_size=(224, 224),
    batch_size=32,
    shuffle=True
)

def split_dataset(dataset, train_size=0.8, val_size=0.1, test_size=0.1):
    total_size = len(dataset)
    train_end = int(total_size * train_size)
    val_end = train_end + int(total_size * val_size)

    train_dataset = dataset.take(train_end)
    val_dataset = dataset.skip(train_end).take(val_end - train_end)
    test_dataset = dataset.skip(val_end)

    return train_dataset, val_dataset, test_dataset

train_dataset, val_dataset, test_dataset = split_dataset(dataset)

Found 2394 files belonging to 6 classes.


In [6]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(6, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [7]:
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom

data_augmentation = tf.keras.Sequential([
    RandomFlip("horizontal"),
    RandomRotation(0.1),
    RandomZoom(0.1)
])

def augment_data(dataset):
    def augment(image, label):
        image = data_augmentation(image)
        return image, label

    return dataset.map(augment)

train_dataset = augment_data(train_dataset)

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20,
    batch_size=32
)

Epoch 1/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 761s 12s/step - accuracy: 0.3802 - loss: 1.6300 - val_accuracy: 0.7812 - val_loss: 0.6656
Epoch 2/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 478s 8s/step - accuracy: 0.7271 - loss: 0.7939 - val_accuracy: 0.8036 - val_loss: 0.5118
Epoch 3/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 493s 8s/step - accuracy: 0.7713 - loss: 0.6429 - val_accuracy: 0.8304 - val_loss: 0.4992
Epoch 4/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 473s 8s/step - accuracy: 0.8130 - loss: 0.5151 - val_accuracy: 0.8661 - val_loss: 0.4013
Epoch 5/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 492s 8s/step - accuracy: 0.8327 - loss: 0.4728 - val_accuracy: 0.8795 - val_loss: 0.3855
Epoch 6/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 475s 8s/step - accuracy: 0.8525 - loss: 0.4210 - val_accuracy: 0.8929 - val_loss: 0.2965
Epoch 7/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 504s 8s/step - accuracy: 0.8666 - loss: 0.3826 - val_accuracy: 0.8795 - val_loss: 0.3532
Epoch 8/20
60/60 ━━━━━━━━━━━━━━━━━━━━ 524s 8s/step - accuracy: 0.8766 - loss: 0.3745 - val_accuracy: 0.9152 - 

In [8]:
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_acc:.4f}")

8/8 ━━━━━━━━━━━━━━━━━━━━ 87s 8s/step - accuracy: 0.9493 - loss: 0.1279
Test Accuracy: 0.9280


In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
# Predictions on test dataset
y_true = []
y_pred = []

for images, labels in test_dataset:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

# Calculate precision, recall, F1-score, and accuracy
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Print the classification report for more details
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=['Acne', 'Carcinoma', 'Eczema', 'Keratosis', 'Milia', 'Rosacea']))

1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
Accuracy: 0.9200
Precision: 0.9237
Recall: 0.9200
F1 Score: 0.9202

Classification Report:
              precision    recall  f1-score   support

        Acne       0.81      0.97      0.89        40
   Carcinoma       0.96      0.96      0.96        49
      Eczema       0.92      0.85      0.88        52
   Keratosis       1.00      0.97      0.99        38
       Milia       0.97      0.94      0.95        33
     Rosacea       0.89      0.84      0.86        38

    accuracy                           0.92       250
   macro avg       0.92      0.92      0.92       250
weighted avg       0.92      0.92      0.92       250

